In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

# Helper Functions

These functions train a model on your data and use cross validation to determine the average negative mean squared error. You do not have to edit these! However, it might make sense to add new helper functions here.

In [2]:
def scoreLinear(X, y):
    """Trains a linear regression model and evaluates it with 3-fold cross validation.
  
    Parameters: 
    X (array): Training data of shape (n_samples, n_features)
    y (array): Target values of shape (n_samples,)
  
    Returns: 
    Float: average score of the model after cross validation
    """
    regr = linear_model.LinearRegression()
    scores = cross_val_score(regr, X, y, cv=3, scoring='neg_mean_squared_error')
    return scores.mean()

In [3]:
def scoreBoosting(X, y):
    """Trains a gradient boosting model and evaluates it with 3-fold cross validation.
  
    Parameters: 
    X (array): Training data of shape (n_samples, n_features)
    y (array): Target values of shape (n_samples,)
  
    Returns: 
    Float: average score of the model after cross validation
    """
    clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=2, random_state=0, loss='squared_error')
    # clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=2, random_state=0, loss='ls') # older versions

    scores = cross_val_score(clf, X, np.ravel(y), cv=3, scoring='neg_mean_squared_error')
    return scores.mean()

In [4]:
def dataframeToXy(df, predict_column, feature_columns):
    """Convert the dataframe to a format usable for the ML algorithms"""
    X = df[feature_columns].values.reshape(-1, df[feature_columns].shape[1]) # all features
    y = df[[predict_column]].values.reshape(-1, 1) # values to predict
    return X, y

In [5]:
def runScoring(df, predict_column, feature_columns):
    """This runs both algorithms to determine the scores for all given features in the dataset.
    
    Parameters: 
    df (dataframe): dataframe with Score and features
    predict_column: name of the value to predict, e.g. 'Score'
    feature_columns: list of all column names that are to be used as features
  
    Returns: 
    linear: negative mean squared error of linear regression
    boost: negative mean squared error of boosting algorithm
    """
    X, y = dataframeToXy(df, predict_column, feature_columns)
    boost = scoreBoosting(X, y)
    linear = scoreLinear(X, y)
    return linear, boost

In [6]:
def runScoringSimple(df, predict_column):
    """Alternative to function above, this takes all the columns in the 
    dataframe as features except the single column which is used for the score.
    """
    all_feature_columns = list(df.columns)
    all_feature_columns.remove(predict_column)
    return runScoring(df, predict_column, all_feature_columns)

In [7]:
def createBaselineRandom(df, predict_column):
    """creates random features and returns the average score after 20 rounds"""
    np.random.seed(0)
    baseline = df[['Score']].copy()
    averageLinearScore = 0
    averageBoostingScore = 0
    rounds = 20
    for i in range(rounds):
        baseline['feature1'] = np.random.randint(0, 100, df.shape[0])
        baseline['feature2'] = np.random.randint(0, 100, df.shape[0])
        baseline['feature3'] = np.random.randint(0, 100, df.shape[0])
        lin, boost = runScoring(baseline, predict_column, ['feature1', 'feature2', 'feature3'])
        averageLinearScore += lin / rounds
        averageBoostingScore += boost / rounds
    print(f'linear   : {averageLinearScore}')
    print(f'boosting : {averageBoostingScore}')

# Load Data

Recommended: Make sure you have a column named 'Score', preferably as the first column, while the name of the municipality is the index. It is fine to use your previous notebook from assignment 1 to export the data to a .csv or excel and simply import the file here, you do not have to show the code for that. Your initial dataframe should look somewhat like the one below, with more features of course.

In [8]:
# TODO: REPLACE THIS WITH YOUR OWN DATA
demodata = {'Gemeinde': ['Neverwinter','Novigrad','Pallet Town','Lothric','Los Santos','City 17','The Citadel','New Vegas', 'Rapture','Grestin'],
        'Score': [90,82,75,50,42,32,15,9,8,2],
        'Luftqualität': [1200, 400, 700, 730, 640, 500, 180, 210, 80, 10]
        }
df = pd.DataFrame(demodata).set_index('Gemeinde')
df

,Score,Luftqualität
Gemeinde,,
Neverwinter,90,1200
Novigrad,82,400
Pallet Town,75,700
Lothric,50,730
Los Santos,42,640
City 17,32,500
The Citadel,15,180
New Vegas,9,210
Rapture,8,80


These are the features that were used to create the original ranking in Assignment 1. Make sure these features are not in your dataset from now on. 
**TODO: Please write down your original features here!**


In [9]:
# TODO: Write down your original features here!

# Get a first baseline for your model with random values

In [10]:
# This code creates a baseline for your model, using random features
createBaselineRandom(df, 'Score')

linear   : -3018.872880199688
boosting : -2526.4952918755994


**WARNING**: This uses the same dataframe for both the linear model and the boosting model. In your code you will want to use different ones for the different models. Your goal is to get both of these scores to as close as 0 as possible and most likely you will need to do different steps for linear regression and for gradient boosting and therefore have different features. 

# Get a second baseline by using all features you have

In [11]:
# this assumes we have a dataframe with a column named Score and all other columns are features.
# You might have to change this code if your dataframe looks different.

linear, boost = runScoringSimple(df, 'Score')
print(f'linear   : {linear}')
print(f'boosting : {boost}')

linear   : -530.3480725643922
boosting : -929.6503901359321


# Extend your features and add something of a geographic nature

Take the features you have loaded above and potentially extend it with the data from the gemeindeporträts 2021. Now you can extend this with some kind of geographic information. This can be pretty much anything that uses coordinates, height, relative position to other features etc. Try to find something reasonable that might help you, but don't worry if you find out during your feature engineering that it is not helpful, you are not required to use the feature for your final model.

In [12]:
# TODO: Insert your code to get a geographic feature here

# Do your own feature engineering here

Your goal is to find the right features to get both scores as close to 0 as possible. Add more columns to your dataframe through feature engineering and choose which ones of those you will use for the machine learning models. Use the methods discussed in class to improve your results even further.

In [13]:
#TODO: Insert your code here 